In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

fullData=[]
for pageNumber in range(1,500):
    print("Getting current page number {}".format(pageNumber))
    initData=requests.get('http://techspy.com/?tab=Home&page={}'.format(pageNumber)).content
    initData=initData.replace('\r','').replace('\n','')
    initsoup = BeautifulSoup(initData, 'html.parser')
    # Getting the initial articles
    articleLinks = initsoup.findAll("div", {"class": "si-content"})
    for curArticle in list(articleLinks):
        innerArticle=list(curArticle.children)[0]['href']
        # Getting the detailed articles
        innerData=requests.get('http://www.techspy.com/' + innerArticle).content
        innerData=innerData.replace('\r','').replace('\n','')
        innersoup = BeautifulSoup(innerData, 'html.parser')
        try:
            title=innersoup.find('h1',{"class":" sd-title"}).text
            article=innersoup.find('p',{"class":" sd-text"}).text
            time=innersoup.find('span',{"class":" sd-time"}).text
            fullData.append([title,article,time])
        except:
            print("Moving on")
    print("After pagenumber {} the length is {}".format(pageNumber,len(fullData)))

In [ ]:
from selenium import webdriver
import os

path_to_phantomJS='/home/ubuntu/anant/SAProject/apps/phantomjs'
browser = webdriver.PhantomJS(executable_path = path_to_phantomJS,service_log_path=os.path.devnull)
browser.get('http://techspy.com/?tab=Home&page=1')
pageContents=browser.page_source
browser.close()
browser.quit()

In [ ]:
fullData=[]
initsoup = BeautifulSoup(pageContents, 'html.parser')
# Getting the initial articles
articleLinks = initsoup.findAll("div", {"class": "si-content"})
for curArticle in list(articleLinks):
    innerArticle=list(curArticle.children)[0]['href']
    # Getting the detailed articles
    browser = webdriver.PhantomJS(executable_path = path_to_phantomJS,service_log_path=os.path.devnull)
    browser.get('http://www.techspy.com/' + innerArticle)
    print('http://www.techspy.com/' + innerArticle)
    innerData=browser.page_source
    browser.close()
    browser.quit()
    innerData=innerData.replace('\r','').replace('\n','')
    innersoup = BeautifulSoup(innerData, 'html.parser')
    try:
        title=innersoup.find('h1',{"class":" sd-title"}).text
        article=innersoup.find('p',{"class":" sd-text"}).text
        time=innersoup.find('span',{"class":" sd-time"}).text
        fullData.append([title,article,time])
    except:
        print("Moving on")
print("After pagenumber {} the length is {}".format(pageNumber,len(fullData)))

In [ ]:
from selenium.webdriver.chrome.options import Options
chromeOptions = Options()
chromeOptions.binary_location='/home/ubuntu/anant/SAProject/apps/chromedriver'
chromeOptions.set_headless(True)
chromeOptions.add_argument('--headless')
chromeOptions.add_argument('--no-sandbox')
chromeOptions.add_argument('--disable-dev-shm-usage')
chromeOptions.add_argument("start-maximized")
chromeOptions.add_argument('--disable-gpu')
chromeOptions.add_argument("disable-infobars")
chromeOptions.add_argument("--disable-extensions")
chromeOptions.experimental_options['useAutomationExtension']=False

In [ ]:
# KNOWTECHIE DATA
#FullData=[]
path_to_phantomJS='/home/ubuntu/anant/SAProject/apps/phantomjs'
path_to_chromeJS='/home/ubuntu/anant/SAProject/apps/chromedriver'
#browser = webdriver.PhantomJS(executable_path = path_to_phantomJS,service_log_path=os.path.devnull)
browser = webdriver.Chrome(chrome_options=chromeOptions,executable_path = path_to_chromeJS)
browser.get('https://knowtechie.com/category/news/')
print('https://knowtechie.com/category/news/')
clickElem=browser.find_element_by_class_name("mvp-inf-more-but")
print(clickElem)
clickElem.click()
clickElem=browser.find_element_by_class_name("mvp-inf-more-but")
print(clickElem)
clickElem.click()
#browser.find_element_by_class_name("mvp-inf-more-but").click()
innerData=browser.page_source
browser.close()
browser.quit()
Data=innerData.replace('\r','').replace('\n','')
Soup = BeautifulSoup(innerData, 'html.parser')
getKnowTechieData(Soup,FullData)

In [ ]:
def getKnowTechieData(Soup,fullData):
    masterElement=Soup.find('ul',{"class" : "mvp-main-blog-wrap left relative infinite-content"})
    for x in masterElement.find_all('li'):
        fullDataText=''
        curText=x.find('h2').text
        curInnerLink=x.find('h2').find('a')['href']
        browser = webdriver.PhantomJS(executable_path = path_to_phantomJS,service_log_path=os.path.devnull)
        browser.get(curInnerLink)
        print("Getting data for {} ".format(curInnerLink))
        innerData=browser.page_source
        browser.close()
        browser.quit()
        innerData=innerData.replace('\r','').replace('\n','')
        innerSoup = BeautifulSoup(innerData, 'html.parser')
        innerTime=innerSoup.find('time',{"class":"post-date updated"}).text
        dataDiv=innerSoup.find('div',{"id":"mvp-content-main"})
        for dataElem in dataDiv.find_all('p'):
            fullDataText= fullDataText +' ' + dataElem.text
        fullData.append([curInnerLink,innerTime,fullDataText]) 

In [ ]:
# Reading the ALL the News Dataset
import pandas as pd
import os
FullData=pd.DataFrame(columns=['id','title','publication','author','date','year','month','url','content'])
def getData(fileName):
    data=[]
    counter=1
    with open('../data/kaggle/all-the-news/{}'.format(fileName),'r') as f1:
        for line in f1:
            if counter > 1:
                curCounter=line[0:line.find(',')]
                line=line[line.find(',')+1:]
                curId=line[0:line.find(',')]
                line=line[line.find(',')+1:]
                line=line.strip()
                if(len(str(line)) > 0):
                    if(line[0]=='"'):
                        line=line[1:]
                        curTitle=line[0:line.find('"')]
                        line=line[line.find('"')+1:]
                        line=line[line.find(',')+1:]
                    else:
                        # Normal Processing
                        curTitle=line[0:line.find(',')]
                        line=line[line.find(',')+1:]
                    curPublication=line[0:line.find(',')]
                    line=line[line.find(',')+1:]
                    curAuthor=line[0:line.find(',')]
                    line=line[line.find(',')+1:]
                    curDate=line[0:line.find(',')]
                    line=line[line.find(',')+1:]
                    curYear=line[0:line.find(',')]
                    line=line[line.find(',')+1:]
                    curMonth=line[0:line.find(',')]
                    line=line[line.find(',')+1:]
                    curURL=line[0:line.find(',')]
                    line=line[line.find(',')+2:]
                    curContent=line
                    data.append([curId,curTitle,curPublication,curAuthor,curDate,curYear,curMonth,curURL,curContent])
            counter=counter+1
    data=pd.DataFrame(data,columns=['id','title','publication','author','date','year','month','url','content'])
    return(data)

for curFileName in os.listdir('../data/kaggle/all-the-news/'):
    print(curFileName)
    FullData=pd.concat([FullData,getData(curFileName)])
    print(FullData.shape)
    
FullData.to_csv('/home/ubuntu/anant/data/allNewsFullData.txt',sep=chr(255),index=False)

In [ ]:
# Read all the news data
# GNF DATA
#!head -5  /home/ubuntu/anant/SAProject/data/kaggle/GNF/file1aa
GNFData=pd.DataFrame(columns=['publish_time','feed_code','source_url','headline_text'])
for curFile in os.listdir('/home/ubuntu/anant/SAProject/data/kaggle/GNF/'):
    print(curFile)
    curData=pd.read_csv('/home/ubuntu/anant/SAProject/data/kaggle/GNF/{}'.format(curFile),skiprows=1,engine='python',sep=',',skip_blank_lines=True,error_bad_lines=False)
    curData.columns=['publish_time','feed_code','source_url','headline_text']
    for curcol in curData.columns:
        curData[curcol]=curData[curcol].map(lambda x : ''.join([y for y in str(x) if ord(y) <= 126 and ord(y) >= 32]))
    GNFData=pd.concat([GNFData,curData])
    print(GNFData.shape)
GNFData.to_csv('/home/ubuntu/anant/data/GNFHeadLineData.txt',sep=chr(255),index=False)

In [7]:
# Double Propagation Algorithm
companyData=pd.read_csv('/home/ubuntu/anant/SAProject/data/CompanyDataSansLocationFull.txt')
#predicateData=pd.read_csv('../data/PredicateDataWithLocationFull.txt.bz2')
#print(predicateData.head(5))

products=companyData[companyData['propertyLabel']=='products']['valueLabel'].unique()
companies=companyData['CompanyLabel'].unique()

       CompanyLabel propertyLabel  \
0      Chuck-A-Rama      abstract   
1                        abstract   
2  Church's Chicken      abstract   
3   Churchs Chicken      abstract   
4   Church & Dwight      abstract   

                                          valueLabel  
0  Chuck-A-Rama is a chain of buffet restaurants ...  
1  Church's Chicken is a U.S.-based chain of fast...  
2  Church's Chicken is a U.S.-based chain of fast...  
3  Church's Chicken is a U.S.-based chain of fast...  
4  Church & Dwight Co, Inc. is a major American m...  
             predicateLabel propertyLabel valueLabel
0  Port_Angeles,_Washington          type      Thing
1  Port_Angeles,_Washington          type      Place
2  Port_Angeles,_Washington          type   Location
3  Port_Angeles,_Washington          type    Q486972
4  Port_Angeles,_Washington          type       Q515


In [86]:
import nltk
import numpy as np
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

def getContinousBigrams(inpArray):
    outArray=[]
    for curElement in inpArray:
        tempArray=curElement.split(' ')
        if(len(tempArray) > 2):
            tempArray
        else:
            outArray.append(curElement)
        
            
            
        

def getDetailedProducts(x,allElements):
    indItems=x.split(',')
    if(len(indItems) > 1):
        indItems=[ ' '.join([str(y).strip().lower() for y in curItem.split(' ') if str(y).lower() not in stopWords]).strip() for curItem in indItems]
        # Now for all the individual items, we will prepare a list of words that we need to search based on importance
        
        allElements[x]=indItems
        #allElements.extend(indItems)
    else:
        indItems=' '.join([str(y).strip().lower() for y in indItems[0].split(' ') if str(y).lower() not in stopWords])
        allElements[x]=[indItems]
        #allElements.append(indItems.strip())
        
allElements={}
[getDetailedProducts(x,allElements) for x in products]
#allElements=list(set(allElements))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [95]:
companyData[companyData['CompanyLabel']=="Church's Chicken"]

,CompanyLabel,propertyLabel,valueLabel
2,Church's Chicken,abstract,Church's Chicken is a U.S.-based chain of fast...
43093,Church's Chicken,foundingDate,1952-04-17
44875,Church's Chicken,foundingYear,1952
64046,Church's Chicken,keyPerson,"George_W._Church,_Sr."
76285,Church's Chicken,location,United_States
76288,Church's Chicken,location,"Sandy_Springs,_Georgia"
105547,Church's Chicken,numberOfLocations,1700
172708,Church's Chicken,industry,Food
177528,Church's Chicken,hypernym,S
412835,Church's Chicken,product,French_fries


In [89]:
allElements

{'Lead': ['lead'],
 'Sandwiches , meals and pizzas': ['sandwiches', 'meals pizzas'],
 'Electronics, Home, Furniture, Outdoor, Sporting Goods, Toys, Jewelry, Clothing, Health and Beauty, Grocery.': ['electronics',
  'home',
  'furniture',
  'outdoor',
  'sporting goods',
  'toys',
  'jewelry',
  'clothing',
  'health beauty',
  'grocery.'],
 'Sugar': ['sugar'],
 'defense vessels': ['defense vessels'],
 'ADINA Thermal': ['adina thermal'],
 'Equipment and systems for water treatment': ['equipment systems water treatment'],
 'contrast agents, injectors, medical devices, pharma': ['contrast agents',
  'injectors',
  'medical devices',
  'pharma'],
 'Motor dealers': ['motor dealers'],
 'AGV': ['agv'],
 'Motorways construction and maintenance': ['motorways construction maintenance'],
 'electricity': ['electricity'],
 'Cellular Telephone Service, Communication Systems': ['cellular telephone service',
  'communication systems'],
 'Print Solutions and Managesd Print Services': ['print solutions 